# Applicative Functors?

Applicative functors are, let me say, a desin pattern, popular in typed functional languages. Mostly Haskell, because of their roots in cateory-theory and that sounds like a thing people writing Haskell like. Based on conference talks Scala people are trying to use this pattern as well. So my question would be, is this still usefull even if I don't have a static type system? 

# Why would anybody deal with Category Theory in programming?

Well, to understand that, you need to imagine yourself in the shoes of a beginner haskell programmer. You have your awesome functional language, and really like the predictability.

* you can't change values
* no null/nil/None/undefined or any such nonsense
* this means there is no global state, awesome
* all functions are pure, stateless, defined only by relation of inputs to outputs, no I/O
 * basically the most testable thing ever

And all of this is awesome, but after a while you realize that not having I/O kind-of sucks, and thar what you have is basically a really tricked-out command-line calculator. (At least that was my experience learning Haskell at my uni)

The question then became, how do we keep the awesome things about Haskell, like no Null-pointer exceptions, testable stateless functions, but add new things on top of it. What if it makes sense to have global variable, I/O, or return null? If want to build i.e. a database, all of these look useful. 

What then happened was that Philip Wadler read a paper about using monads from category theory as interface to chain I/O operations. And he implemented this as the base for I/O in haskell. And 25 years later we see monadic interfaces in C# Linq and Tasks, Java Streams and almost in Javascript Promise.

I do want to explain wat category theory *is* and how does it map to programming.

# Introducing Functor

Category theory is usually concerned with relation between diferent categories, but for programming, we are usually concerned only with category of types and the unary functions that convert between them.

When programmer talks about relarion between two categories,
usually one will basically be his favourite language and second one
some fancy extension to that language.

Like said original Haskell and Haskell with I/O.

So what in practice is a fuctor?

In general, anything where you can define a sensible `map` operation.
For example:

* list
* stream
* generator
* tree

Sensible for list means, that it shouldn't matter whether you
strs.map(trim).map(uppercase)

or

strs.map(lambda x: uppercase(trim(x)))

# Do you even lift?

There is a recuring concept, where we use some bit of information about
our datatype to convert functions that know nothig about it, to work with it.

We call this *lifting*.

For example, because I know that lists have map function, I could lift i.e. `str` to work on lists of things.


In [2]:
def lift(fn):
    def liftedfn(x):
      return list(map(fn,x))
    return liftedfn 

In [3]:
lifted_str = lift(str)
lifted_str([1,2,3])

['1', '2', '3']

# What if I have multiple parameters?

Mapping only single param functions can be limiting. It would be cool if `lift` would could work on function with arbitrary many parameters.

Could *currying* rescue us?

Currying is a concept, where instead of passing in all the params into the function at once, you pass in the first param and grt a new function as a result. This function accepts the second param and returns a function that accepts the third, and by now you should be getting a picture.

Or maybe I should show an example.Lets invent a nontrivial 3 arg function, we will call it erm, `window` (because slice was taken).

In [4]:
from functools import partial

def window(begin,end,array):
    return array[begin:end]

window(2,4,[1,2,3,4,5])

[3, 4]

If this function were curried, it would look like this:

In [5]:
def curried_window(begin):
    return lambda end: (lambda array: array[begin:end])
curried_window(2)(4)([1,2,3,4,5])

[3, 4]

Some functional languages have this as a default. The nice thing about that is, that you can quickly create more specific fuctions, without the need to wrap them in new function definitions.

And this seems to solve out problem with multi param function, because sudenly, everything is a single param function, if you squint hard enough :)

In [7]:
list(map(curried_window,[1,2,3,4]))

[<function __main__.curried_window.<locals>.<lambda>>,
 <function __main__.curried_window.<locals>.<lambda>>,
 <function __main__.curried_window.<locals>.<lambda>>,
 <function __main__.curried_window.<locals>.<lambda>>]

Ok, as you can see, currying won't save us just yet, because even though we can use map with them,
result is kind-of useless list of functions. We would need to somehow be able to *apply* this list of functions on a list of values. 

In [10]:
def apply(lfn,lx):
    return [fn(x) for fn,x in zip(lfn,lx)]


a0 = map(curried_window,[1,2,3,4])
a1 = apply(a0,[3,4,5,6])
a2 = apply(a1,[
    [1,2,3,4,5,6],
    [1,2,3,4,5,6],
    [1,2,3,4,5,6],
    [1,2,3,4,5,6]
              ])

a2

[[2, 3], [3, 4], [4, 5], [5, 6]]

Yay, this seems to have worked :)

Now we can try to combine currying with *apply* and create a propper lift function :)
